# Введение в обработку естественного языка
## Урок 3. Embedding word2vec fasttext
### Скачиваем датасет через ноутбук

In [ ]:
!wget https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
!wget https://www.dropbox.com/s/r6u59ljhhjdg6j0/negative.csv
!pip install apache_beam
!pip install 'scikit_learn~=0.23.0'
!pip install annoy
!pip install pymorphy2
!pip install stop_words

--2022-06-03 21:46:17--  https://www.dropbox.com/s/fnpq3z4bcnoktiv/positive.csv
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601d:18::a27d:512
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /s/raw/fnpq3z4bcnoktiv/positive.csv [following]
--2022-06-03 21:46:18--  https://www.dropbox.com/s/raw/fnpq3z4bcnoktiv/positive.csv
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://ucfd90950f0934265b054169bbe3.dl.dropboxusercontent.com/cd/0/inline/BmgjufSLG5xuvWNquFQvsJHfPKMhanaSfJlzYTqlLr5Rlrj245N8pTdWYO24NyvclHHDm0pJQ1TLYFUQwZiQH_NG4eji6vTQELe4daUU3hOFmM0jCL7cCdVOK4N20kQn3rNgfii9iCYDSMA7mLuXlhKI1KCoyVZohG3HWOCr5Bm-Gg/file# [following]
--2022-06-03 21:46:18--  https://ucfd90950f0934265b054169bbe3.dl.dropboxusercontent.com/cd/0/inline/BmgjufSLG5xuvWNquFQvsJHfPKMhanaSfJlzYTqlLr5Rlrj245N8pTdWYO24NyvclHHD

In [ ]:
import os
import urllib.request
from scipy import spatial
import os
import urllib.request
from scipy import spatial
import pandas as pd
import numpy as np
import codecs
from sklearn.metrics import *
from sklearn.model_selection import train_test_split
import re
from html.parser import HTMLParser
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from string import punctuation
from pymystem3 import Mystem
from gensim.models import Word2Vec, FastText
import annoy
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from string import punctuation
nltk.download('stopwords')
stop_words = set(stopwords.words("russian"))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1. объединить в одну выборку

In [ ]:
positive = pd.read_csv('positive.csv', sep=';', usecols=[3], names=['text'])
positive['label'] = ['positive'] * len(positive)
negative = pd.read_csv('negative.csv', sep=';', usecols=[3], names=['text'])
negative['label'] = ['negative'] * len(negative)
df = pd.concat([positive, negative])
df = df.reset_index().drop('index', axis=1)

### 2. на основе word2vec/fasttext/glove/слоя Embedding реализовать метод поиска ближайших твитов
(на вход метода должен приходить запрос (какой-то твит, вопрос) и количество вариантов вывода к примеру 5-ть, ваш метод должен возвращать 5-ть ближайших твитов к этому запросу)

In [ ]:
def preprocess(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [ ]:
print(df['text'].values)

['@first_timee хоть я и школота, но поверь, у нас то же самое :D общество профилирующий предмет типа)'
 'Да, все-таки он немного похож на него. Но мой мальчик все равно лучше:D'
 'RT @KatiaCheh: Ну ты идиотка) я испугалась за тебя!!!' ...
 'Вот и в школу, в говно это идти уже надо('
 'RT @_Them__: @LisaBeroud Тауриэль, не грусти :( *обнял*'
 'Такси везет меня на работу. Раздумываю приплатить, чтобы меня втащили на пятый этаж. Лифта то нет :(']


In [ ]:
sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("ru"))
exclude = set(punctuation)
for line in df['text'].values:
    spls = preprocess(line)
    sentences.append(spls)

In [ ]:
modelW2V = Word2Vec(sentences=df['text'].values, size=300, window=5, min_count=1)
modelFT = FastText(sentences=df['text'].values, size=300, min_count=1, window=5, workers=8)

w2v_index = annoy.AnnoyIndex(300 ,'angular')
ft_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for line in df['text'].values:
    if len(line) > 0:
        n_w2v = 0
        n_ft = 0
        spls = line.split()

        question = preprocess(spls[0])

        vector_w2v = np.zeros(300)
        vector_ft = np.zeros(300)
        for word in question:
            if word in modelW2V.wv:
                vector_w2v += modelW2V.wv[word]
                n_w2v += 1
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_w2v > 0:
            vector_w2v = vector_w2v / n_w2v
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        w2v_index.add_item(counter, vector_w2v)
        ft_index.add_item(counter, vector_ft)

        counter += 1


w2v_index.build(10)
ft_index.build(10)

True

In [ ]:
!pip install --upgrade scikit-learn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.8 MB 2.0 MB/s 
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 0.23.2
    Uninstalling scikit-learn-0.23.2:
      Successfully uninstalled scikit-learn-0.23.2


In [ ]:
from sklearn.manifold import TSNE
import spacy

In [ ]:
nlp = spacy.load('en_core_web_sm')

In [ ]:
emmbed_dict = {}
for line in df['text'].values:
  values = nlp(line)
  for w in values:
    word = w
    vector = w.vector
    emmbed_dict[word]=vector

In [ ]:
def find_similar_word(emmbedes):
  nearest = sorted(emmbed_dict.keys(), key=lambda word: spatial.distance.euclidean(emmbed_dict[word], emmbedes))
  return nearest

In [ ]:
modelGlove = TSNE(n_components=2)
words = list(emmbed_dict.keys())
vectors = [emmbed_dict[word] for word in words]
mGlove = modelGlove.fit_transform(vectors)

mG_index = annoy.AnnoyIndex(300 ,'angular')

index_map = {}
counter = 0

for line in df['text'].values:
    if len(line) > 0:
        n_mG = 0
        spls = line.split()

        question = preprocess(spls[0])

        vector_mG = np.zeros(300)
        for word in question:
            if word in mGlove.wv:
                vector_ += mGlove.wv[word]
                n_mG += 1
        if n_mG > 0:
            vector_mG = vector_mG / n_mG
        mG_index.add_item(counter, vector_mG)

        counter += 1


mG_index.build(10)

### 3. Проверить насколько хорошо работают подходы

In [ ]:
def get_response(question, index, model):
    question = preprocess(question)
    vector = np.zeros(300)
    norm = 0
    for word in question:
        if word in model.wv:
            vector += model.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    return index.get_nns_by_vector(vector, 5)

In [ ]:
quest = 'как учеба в гикбрейнс'

df.loc[get_response(quest, w2v_index, modelW2V)]

,text,label
789,Carnival Phantasm всё! Адов ёбаный пиздец! Ржа...,positive
1474,Какие снежные хлопья за окном. Бел лит. Такая ...,positive
2714,RT @Azad_341: Разобрался наконец то как управл...,positive
2924,"@canterra2 Да,у нас таких и не сыщешь...западн...",positive
3073,"@Art_mast а я тебе еще раз говорю, что пидорст...",positive


In [ ]:
df.loc[get_response(quest, ft_index, modelFT)]

,text,label
1748,Вот это понимание у нас с эл) http://t.co/8YSu...,positive
1754,бомбита)))))))))))))))))))))))))))))))))))))))...,positive
2230,"@_yeeeah_ не поняла к чему ты это написала, ес...",positive
3281,RT @alina_0101: Азаза.только я понял:D http://...,positive
5420,Завтра будем измерять глубину бассейна)) #ахах...,positive


Выводы:  
- не понял алгоритм Glove;  
- word2vec и fasttext сработали посредственно, так как подобранные твиты нельзя назвать релевантными.